In [1]:
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import logging
import os


import pyro
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO, MCMC, NUTS
import pyro.distributions as dist
import pyro.distributions.constraints as constraints


pyro.__version__

'1.8.4'

# Latent Dirichlet Algorithm

extract valid subjects

In [3]:
from fear_gen import extract_correct_csv
os.chdir("fear_gen")
valid_sub = extract_correct_csv.extract_only_valid_subject()

[1,
 2,
 4,
 5,
 6,
 7,
 13,
 15,
 16,
 17,
 21,
 22,
 23,
 26,
 27,
 28,
 31,
 32,
 33,
 41,
 43,
 44,
 45,
 46,
 47,
 48,
 50,
 51,
 52,
 54]

read real data of subject #2

In [4]:
# read dataset
os.chdir('..')
df = pd.read_csv('fear_gen/data/newLookAtMe/newLookAtMe02.csv')
df_rational = df[['morphing level', 'shock']]
df_rational['shock'] = df_rational['shock'].astype(int) #setting shock as int instead of boolean
df_rational['morphing level'] = [int(d==6) for d in df_rational['morphing level']] # if morphing level==6 -> 1

/var/folders/hn/nq1_cby521sbf58lnj9qj9mw0000gn/T/ipykernel_14953/76822954.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rational['shock'] = df_rational['shock'].astype(int)
/var/folders/hn/nq1_cby521sbf58lnj9qj9mw0000gn/T/ipykernel_14953/76822954.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rational['morphing level'] = [int(d==6) for d in df_rational['morphing level']]


In [5]:
# categorical/multinomial distribution

# uniform prior
prior_counts = torch.ones((2,2))

def model(data):
    prior = pyro.sample("prior", dist.Dirichlet(prior_counts))
    total_counts = int(data.sum())
    pyro.sample("posterior", dist.Multinomial(total_counts, prior), obs=data)

In [6]:
data_np = df_rational.to_numpy()
data_all = data_np[16:]
learning_data = data_np[16:48] # considering only learning trials
learning_data

array([[1, 1],
       [0, 0],
       [0, 0],
       [1, 1],
       [0, 0],
       [1, 1],
       [1, 1],
       [0, 0],
       [1, 0],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [1, 0],
       [0, 0],
       [1, 1],
       [1, 1],
       [0, 0],
       [1, 0],
       [0, 0],
       [1, 1],
       [0, 0],
       [1, 0],
       [0, 0],
       [1, 1],
       [0, 0],
       [1, 1],
       [0, 0],
       [1, 1],
       [0, 0]])

In [7]:
data = torch.tensor(data_all)
N = data.shape[0]
dict_ = {'[0 0]':0, '[0 1]': 0, '[1 0]':0, '[1 1]':0}
for x in data_all:
    dict_[str(x)] += 1


In [8]:
counter = torch.zeros((N,4))

for i in range(len(data_all)):
    dict_ = {'[0 0]':0, '[0 1]': 0, '[1 0]':0, '[1 1]':0}
    tmp_data = data_all[:i+1]
    for x in tmp_data:
        dict_[str(x)] += 1
    values = np.array(list(dict_.values()))
    counter[i] = torch.tensor(values)

In [9]:
counter = counter.reshape((len(data_all), 2, 2))
counter

tensor([[[ 0.,  0.],
         [ 0.,  1.]],

        [[ 1.,  0.],
         [ 0.,  1.]],

        [[ 2.,  0.],
         [ 0.,  1.]],

        [[ 2.,  0.],
         [ 0.,  2.]],

        [[ 3.,  0.],
         [ 0.,  2.]],

        [[ 3.,  0.],
         [ 0.,  3.]],

        [[ 3.,  0.],
         [ 0.,  4.]],

        [[ 4.,  0.],
         [ 0.,  4.]],

        [[ 4.,  0.],
         [ 1.,  4.]],

        [[ 4.,  0.],
         [ 1.,  5.]],

        [[ 5.,  0.],
         [ 1.,  5.]],

        [[ 6.,  0.],
         [ 1.,  5.]],

        [[ 7.,  0.],
         [ 1.,  5.]],

        [[ 8.,  0.],
         [ 1.,  5.]],

        [[ 8.,  0.],
         [ 1.,  6.]],

        [[ 8.,  0.],
         [ 2.,  6.]],

        [[ 9.,  0.],
         [ 2.,  6.]],

        [[ 9.,  0.],
         [ 2.,  7.]],

        [[ 9.,  0.],
         [ 2.,  8.]],

        [[10.,  0.],
         [ 2.,  8.]],

        [[10.,  0.],
         [ 3.,  8.]],

        [[11.,  0.],
         [ 3.,  8.]],

        [[11.,  0.],
         [ 

In [10]:
nuts_kernel = NUTS(model)
num_samples, warmup_steps = (300, 200)

mcmc = MCMC(nuts_kernel, num_samples=num_samples, warmup_steps=warmup_steps, disable_progbar=True)
all_means = []

for i in range(len(counter)):
    mcmc.run(counter[i])
    hmc_samples = {k: v.detach().cpu().numpy()
                for k, v in mcmc.get_samples().items()}
    means = hmc_samples['prior'].mean(axis=0)
    stds = hmc_samples['prior'].std(axis=0)
    print('observation: ', data_all[i])
    print('probabilities: ', means)
    all_means.append(means)

observation:  [1 1]
probabilities:  [[0.51642144 0.48357844]
 [0.32457966 0.6754207 ]]
observation:  [0 0]
probabilities:  [[0.6729735  0.32702678]
 [0.32506058 0.67493945]]
observation:  [0 0]
probabilities:  [[0.7431099  0.25689012]
 [0.322599   0.67740107]]
observation:  [1 1]
probabilities:  [[0.7372084  0.26279148]
 [0.26225013 0.7377501 ]]
observation:  [0 0]
probabilities:  [[0.7999909  0.20000905]
 [0.2730125  0.7269871 ]]
observation:  [1 1]
probabilities:  [[0.7859121  0.2140879 ]
 [0.20459197 0.79540795]]
observation:  [1 1]
probabilities:  [[0.8001468  0.19985338]
 [0.16650237 0.83349764]]
observation:  [0 0]
probabilities:  [[0.8398203  0.16017935]
 [0.16020943 0.83979064]]
observation:  [1 0]
probabilities:  [[0.83040905 0.16959079]
 [0.2726035  0.7273967 ]]
observation:  [1 1]
probabilities:  [[0.836752   0.16324833]
 [0.28405067 0.7159494 ]]
observation:  [0 0]
probabilities:  [[0.8572988  0.14270076]
 [0.26824936 0.7317506 ]]
observation:  [0 0]
probabilities:  [[0.883

KeyboardInterrupt: 

In [ ]:
X = np.arange(16, 16+len(all_means))
X_training = np.arange(16, 48)
all_means = np.array(all_means)
y = all_means[:, 1, 1]
y_training = all_means[:32, 1, 1]

In [ ]:
list_ = []
for index, i in enumerate(data_all):
    if np.equal(i, np.array([1,1])).all():
        list_.append(np.array([X[index], y[index]]))
x_points, y_points = np.array(list_)[:,0], np.array(list_)[:,1]


In [ ]:
fig = plt.figure(figsize=(12,8))

plt.plot(X, y, label='P(C=1|S=1)', color = 'black', alpha=0.8)
plt.scatter(x_points, y_points, marker='*', color='red', linewidths=2, label='CS+ and shock')
plt.ylabel('p')
plt.xlabel('Trial')
plt.legend()
plt.axvline(x=48, linestyle='--')
plt.axvline(x=16, linestyle='--')
plt.show()

In [ ]:
array_csplus = []
array_csminus = []
for index, data in enumerate(data_all):
    if data[0] == 1:
        array_csplus.append([X[index], all_means[index,1,1]])
    else:
        array_csminus.append([X[index], all_means[index,0,1]])

csplus_learning = []
csminus_learning = []
total_simulated = []
for index_train, data_train in enumerate(learning_data):
    if data_train[0] == 1:
        csplus_learning.append([X_training[index_train], all_means[index_train,1,1]])
        total_simulated.append(all_means[index_train,1,1])
    else:
        csminus_learning.append([X_training[index_train], all_means[index_train,0,1]])
        total_simulated.append(all_means[index_train,0,1])

total_simulated = np.array(total_simulated)
array_csplus = np.array(array_csplus)
array_csminus = np.array(array_csminus)
csplus_learning = np.array(csplus_learning)
csminus_learning = np.array(csminus_learning)

In [ ]:
len(total_simulated)

In [ ]:
print(array_csminus[:,0], csminus_learning[:,0])

In [ ]:
df_ = df[['shock', 'rating', 'morphing level']]
df_['shock'] = df_['shock'].astype(int)
df_['morphing level'] = [int(d == 6) for d in df_['morphing level']]
df_['rating'] = df_['rating'].replace([1, 2, 3, 4, 5], [0.2, 0.4, 0.6, 0.8, 1])

In [ ]:
df_new=df_[16:]
df_learning = df_[16:48]
df_learning

In [ ]:
df_learning

In [ ]:
y_plus_real = df_new.loc[df_new['morphing level']==1].rating.values
x_plus_real = np.array(df_new.loc[df_new['morphing level']==1].index)
y_minus_real = df_new.loc[df_new['morphing level']==0].rating.values
x_minus_real = np.array(df_new.loc[df_new['morphing level']==0].index)
y_real=df_new.rating.values

y_plus_train = df_learning.loc[df_learning['morphing level']==1].rating.values
x_plus_train = np.array(df_learning.loc[df_learning['morphing level']==1].index)
y_minus_train = df_learning.loc[df_learning['morphing level']==0].rating.values
x_minus_train = np.array(df_learning.loc[df_learning['morphing level']==0].index)
y_train=df_learning.rating.values

In [ ]:
fig = plt.figure(figsize=(13,6))
plt.scatter(x_plus_real, y_plus_real, color='blue', label='cs+ real')
plt.scatter(x_minus_real, y_minus_real, color='red', label='non cs+ real')
plt.scatter(array_csplus[:,0], array_csplus[:,1], color='darkblue', label='cs+ simulated')
plt.scatter(array_csminus[:,0], array_csminus[:,1], color='darkred', label='cs- simulated')
plt.legend(loc='right')
plt.axvline(x=16, linestyle='--', color='green')
plt.axvline(x=48, linestyle='--', color='green')
plt.xlabel('Trial')
plt.ylabel('P(condition | visual stimulus)')
plt.show()

In [ ]:
print(len(y_minus_train), len(csminus_learning[:,1]))

In [ ]:
fig2 = plt.figure(figsize=(13,6))
plt.scatter(x_plus_train, y_plus_train, color='blue', label='cs+ real')
plt.scatter(x_minus_train, y_minus_train, color='red', label='non cs+ real')
plt.scatter(csplus_learning[:,0], csplus_learning[:,1], color='darkblue', label='cs+ simulated')
plt.scatter(csminus_learning[:,0], csminus_learning[:,1], color='darkred', label='cs- simulated')
plt.legend(loc='right')
plt.axvline(x=16, linestyle='--', color='green')
plt.axvline(x=48, linestyle='--', color='green')
plt.xlabel('Trial')
plt.ylabel('P(condition | visual stimulus)')
plt.show()
plt.show()

In [ ]:
y_plus_train

In [ ]:
csplus_learning[:,1]

In [ ]:
np.corrcoef(y_plus_train, csplus_learning[:,1])

In [ ]:
np.corrcoef(y_minus_train, csminus_learning[:,1])

In [ ]:
np.corrcoef(y_train,total_simulated)

In [ ]:
total_simulated